In [1]:
import torch.nn as nn
import torch
import sys
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import string
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
#Membuka file data opini pilkada DKI
data_pilkada = pd.read_csv("your file")

In [4]:
data_pilkada['Sentiment'].value_counts()

positive    450
negative    450
Name: Sentiment, dtype: int64

In [178]:
data_pilkada

,Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
...,...,...,...,...
895,896,positive,Anies-Sandi,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
896,897,positive,Anies-Sandi,Kita harus dapat merangkul semua orang tanpa b...
897,898,positive,Anies-Sandi,Ini jagoanku dibidang digital <Smiling Face Wi...
898,899,positive,Anies-Sandi,#PesanBijak #OkeOce #GubernurGu3 ...


In [3]:
#Proses filter
def filtering_text(text):
    # mengubah tweet menjadi huruf kecil
    text = text.lower()
    # menghilangkan url
    text = re.sub(r'https?:\/\/\S+','',text)
    # menghilangkan mention, link, hastag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # menghilangkan yang bukan huruf
    text = re.sub('[^a-zA-Z]', ' ', text)
    #menghilangkan tanda baca
    text = text.translate(str.maketrans("","",string.punctuation))
    # menghilangkan whitespace berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data_pilkada['filtered'] = data_pilkada['Text Tweet'].apply(filtering_text)
print(data_pilkada['filtered'].head())

0    banyak akun kloning seolah pendukung mulai men...
1    bicara apa kasihan yaa lap itu air matanya wkw...
2    kalau aku sih gak nunggu hasil akhir qc tp lag...
3    kasian oh kasian dengan peluru milyar untuk ti...
4              maaf ya pendukung hayo dukung diputaran
Name: filtered, dtype: object


In [4]:
#Positive = 1, negative = 0
data_pilkada = data_pilkada.replace({'Sentiment': {'positive': 1, 'negative': 0}})

In [181]:
data_pilkada

,Id,Sentiment,Pasangan Calon,Text Tweet,filtered
0,1,0,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...,banyak akun kloning seolah pendukung mulai men...
1,2,0,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...,bicara apa kasihan yaa lap itu air matanya wkw...
2,3,0,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...,kalau aku sih gak nunggu hasil akhir qc tp lag...
3,4,0,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...,kasian oh kasian dengan peluru milyar untuk ti...
4,5,0,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...,maaf ya pendukung hayo dukung diputaran
...,...,...,...,...,...
895,896,1,Anies-Sandi,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ...",kali saja bpk lihat lagi di rspun tetap selfie...
896,897,1,Anies-Sandi,Kita harus dapat merangkul semua orang tanpa b...,kita harus dapat merangkul semua orang tanpa b...
897,898,1,Anies-Sandi,Ini jagoanku dibidang digital <Smiling Face Wi...,ini jagoanku dibidang digital smiling face wit...
898,899,1,Anies-Sandi,#PesanBijak #OkeOce #GubernurGu3 ...,


In [5]:
data_pilkada = data_pilkada[['Sentiment', 'filtered']]

In [183]:
data_pilkada

,Sentiment,filtered
0,0,banyak akun kloning seolah pendukung mulai men...
1,0,bicara apa kasihan yaa lap itu air matanya wkw...
2,0,kalau aku sih gak nunggu hasil akhir qc tp lag...
3,0,kasian oh kasian dengan peluru milyar untuk ti...
4,0,maaf ya pendukung hayo dukung diputaran
...,...,...
895,1,kali saja bpk lihat lagi di rspun tetap selfie...
896,1,kita harus dapat merangkul semua orang tanpa b...
897,1,ini jagoanku dibidang digital smiling face wit...
898,1,


In [6]:
data_pilkada = data_pilkada.rename(columns={"Sentiment": "label", "filtered": "text"})

In [185]:
data_pilkada

,label,text
0,0,banyak akun kloning seolah pendukung mulai men...
1,0,bicara apa kasihan yaa lap itu air matanya wkw...
2,0,kalau aku sih gak nunggu hasil akhir qc tp lag...
3,0,kasian oh kasian dengan peluru milyar untuk ti...
4,0,maaf ya pendukung hayo dukung diputaran
...,...,...
895,1,kali saja bpk lihat lagi di rspun tetap selfie...
896,1,kita harus dapat merangkul semua orang tanpa b...
897,1,ini jagoanku dibidang digital smiling face wit...
898,1,


In [7]:
from itertools import chain

In [8]:
y = data_pilkada['label']
X = data_pilkada['text']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [10]:
from nltk import TweetTokenizer

In [11]:
tokenizer = TweetTokenizer()
train_data = []
for i,j in zip(X_train, y_train):
    token = tokenizer.tokenize(i)
    label = j    
    train_data.append((token, label))

In [289]:
train_data

[(['dammnn', 'politik', 'cantik', 'ngorbanin'], 0),
 (['jika',
   'pkb',
   'dukung',
   'di',
   'putaran',
   'ke',
   'maka',
   'siap',
   'siap',
   'saja',
   'ditinggalkan',
   'pemilih',
   'umat',
   'islam',
   'di',
   'pemilu'],
  0),
 (['saya',
   'putaran',
   'pertama',
   'dukung',
   'ibu',
   'dan',
   'mas',
   'agus',
   'yah',
   'lebih',
   'baik',
   'kalah',
   'dengan',
   'terhormat',
   'daripada',
   'menang',
   'karena',
   'curang'],
  1),
 (['menang',
   'pendukungnya',
   'puji',
   'syukur',
   'kalah',
   'pendukungnya',
   'nangis',
   'gak',
   'kbayang',
   'kalo',
   'anies',
   'kalah',
   'apa',
   'ada',
   'aksi',
   'demo',
   'lagi'],
  0),
 (['langsung',
   'deh',
   'cuitan',
   'curang',
   'lah',
   'black',
   'campaign',
   'lah',
   'nongol',
   'katanya',
   'legowo',
   'itu',
   'demokrasi',
   'bukan',
   'namanya',
   'hahaa'],
  0),
 (['kata',
   'babe',
   'pilih',
   'nomor',
   'victory',
   'hand',
   'victory',
   'hand',
 

In [12]:
test_data = []
for i,j in zip(X_test,y_test):
    token = tokenizer.tokenize(i)
    label = j
    test_data.append((token,label))

In [291]:
test_data

[(['ahy',
   'dan',
   'tim',
   'berjalan',
   'menjauh',
   'dari',
   'hiruk',
   'pikuk',
   'pilkada',
   'melanjutkan',
   'perjalanan',
   'panjang',
   'politik',
   'mereka',
   'dengan',
   'rasa',
   'kecewa'],
  0),
 (['waduh',
   'maaf',
   'saya',
   'ahoker',
   'tapi',
   'kalo',
   'yang',
   'ini',
   'saya',
   'setuju',
   'sama',
   'pak',
   'anies',
   'gimana',
   'ya'],
  1),
 (['dan', 'kita', 'tetap'], 1),
 (['siaran',
   'langsung',
   'pak',
   'ahok',
   'grining',
   'face',
   'with',
   'smile',
   'eye'],
  1),
 (['pakai', 'sedotan', 'atau', 'sodetan', 'versi'], 0),
 (['isis',
   'juga',
   'lagi',
   'proses',
   'hukum',
   'terus',
   'knapa',
   'komentari',
   'grining',
   'face',
   'grining',
   'face',
   'grining',
   'face',
   'males',
   'betengkar',
   'sama',
   'anak',
   'tk',
   'gak',
   'ber',
   'bobot',
   'asal',
   'ngeles',
   'person',
   'raising',
   'hand',
   'face',
   'with',
   'stuck',
   'out',
   'tongue',
   'closed'

In [15]:
train_data[:2]

[(['dammnn', 'politik', 'cantik', 'ngorbanin'], 0),
 (['jika',
   'pkb',
   'dukung',
   'di',
   'putaran',
   'ke',
   'maka',
   'siap',
   'siap',
   'saja',
   'ditinggalkan',
   'pemilih',
   'umat',
   'islam',
   'di',
   'pemilu'],
  0)]

In [193]:
train_data[4]

(['langsung',
  'deh',
  'cuitan',
  'curang',
  'lah',
  'black',
  'campaign',
  'lah',
  'nongol',
  'katanya',
  'legowo',
  'itu',
  'demokrasi',
  'bukan',
  'namanya',
  'hahaa'],
 0)

In [16]:
test_data[:2]

[(['ahy',
   'dan',
   'tim',
   'berjalan',
   'menjauh',
   'dari',
   'hiruk',
   'pikuk',
   'pilkada',
   'melanjutkan',
   'perjalanan',
   'panjang',
   'politik',
   'mereka',
   'dengan',
   'rasa',
   'kecewa'],
  0),
 (['waduh',
   'maaf',
   'saya',
   'ahoker',
   'tapi',
   'kalo',
   'yang',
   'ini',
   'saya',
   'setuju',
   'sama',
   'pak',
   'anies',
   'gimana',
   'ya'],
  1)]

In [13]:
all_token = list(set(chain.from_iterable([i[0] for i in train_data])))
len(all_token)

2669

In [18]:
all_token

['tahu',
 'pagi',
 'adjrot',
 'putaran',
 'tujuannya',
 'mas',
 'djagain',
 'moga',
 'moto',
 'ntb',
 'tanah',
 'bngga',
 'sajalah',
 'n',
 'jawaban',
 'penjaja',
 'suaramu',
 'ucapan',
 'bbrp',
 'tunggu',
 'dipimpin',
 'pesonanya',
 'bersalah',
 'tetapi',
 'maksa',
 'pancasila',
 'lovered',
 'great',
 'santri',
 'keramaian',
 'seneng',
 'pd',
 'simpatik',
 'skeptis',
 'digunakan',
 'benar',
 'isu',
 'taruna',
 'astaga',
 'keberhasilan',
 'kulon',
 'buruh',
 'lihatnya',
 'nongkrong',
 'ibu',
 'taufik',
 'jalanmu',
 'pilgub',
 'terbang',
 'kemudian',
 'kualitas',
 'tjahaja',
 'bismillah',
 'dituduh',
 'kinerja',
 'merayakan',
 'kampanye',
 'manggung',
 'bijaksana',
 'persaingan',
 'memajukan',
 'belakangnya',
 'komplek',
 'iklan',
 'bocah',
 'bagikan',
 'diaplikasikan',
 'orang',
 'sebelumnya',
 'protes',
 'kelompok',
 'loyo',
 'pdhal',
 'precariat',
 'menginspiras',
 'mirip',
 'hdapkan',
 'usai',
 'bakal',
 'martin',
 'improvisasi',
 'sumur',
 'selamat',
 'rejeki',
 'apresiasi',
 'maha

In [14]:
word2index = {w:i for i, w in enumerate(all_token, start=1)}
word2index['_UNK_'] = len(word2index)
word2index['_PAD_'] = 0

In [15]:
def token2seq(token):
    return [word2index.get(t, word2index['_UNK_']) for t in token]

In [16]:
train_set = [(token2seq(l[0]), l[1]) for l in train_data]
test_set = [(token2seq(l[0]), l[1]) for l in test_data]

In [17]:
train_set[:1]

[([87, 2276, 2009, 436], 0)]

In [18]:
maxlen = 20
def padding(seq):
    if len(seq) > maxlen:
        seq = seq[:maxlen]
    elif len(seq) < maxlen:
        seq = seq + [word2index['_PAD_']] * (maxlen-len(seq))
    return seq

In [19]:
padding(train_set[0][0])

[87, 2276, 2009, 436, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [21]:
train_data_padded = [(padding(l[0]), l[1]) for l in train_set]
test_data_padded = [(padding(l[0]), l[1]) for l in test_set]

In [22]:
train_data_padded[:1]

[([87, 2276, 2009, 436, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0)]

In [23]:
test_data_padded[:1]

[([1563,
   1681,
   1072,
   1399,
   2669,
   1310,
   2669,
   2669,
   64,
   2576,
   2669,
   2605,
   2276,
   2238,
   2594,
   1798,
   2489,
   0,
   0,
   0],
  0)]

In [57]:
batch_size = 10
def batching(data):
    batch_list = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        x = torch.LongTensor([l[0] for l in batch])
        y = torch.LongTensor([l[1] for l in batch])
        batch_list.append((x, y))
    return batch_list

In [39]:
batching(train_data_padded)

[(tensor([[1451,  179, 1215, 1725,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0],
          [ 233,  661, 2063, 1547,    4, 2241, 1450,  880,  880,  914, 1853, 1621,
           1641,  606, 1547, 2081,    0,    0,    0,    0],
          [1555,    4, 2233, 2063,   45, 2015,    6, 2514,  875,  400,  631, 1684,
           1084, 1720,  460, 1625, 1431,  441,    0,    0],
          [1625, 1211, 1872, 1389, 1684, 1211, 1103,  765, 2260, 1955,  105, 1684,
           1648,  957,  494, 2376, 1360,    0,    0,    0],
          [1325, 2625, 2209,  441,  760,  897,  576,  760, 2312,  795, 2380,  299,
           1008, 1777, 2359, 1635,    0,    0,    0,    0],
          [2053,  140, 1066, 1804, 1119, 1653, 1119, 1653, 1119, 1653,  868, 2368,
            704,    0,    0,    0,    0,    0,    0,    0],
          [1506, 1613, 1349,  532,  630, 1390,  978,  383, 2481, 1452, 2291,    0,
              0,    0,    0,    0,    0,    0,    0,    0],

In [59]:
train_batch = batching(train_data_padded)
test_batch = batching(test_data_padded)
test_batch

[(tensor([[1563, 1681, 1072, 1399, 2669, 1310, 2669, 2669,   64, 2576, 2669, 2605,
           2276, 2238, 2594, 1798, 2489,    0,    0,    0],
          [ 544,  459, 2105, 2385, 2501,  932, 1493, 2611, 2105, 2297,  336, 1753,
            917, 2669, 2622,    0,    0,    0,    0,    0],
          [1681,  888, 1352,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0],
          [2669,   75, 1753,  312,  670, 2651, 2637, 1734, 2081,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0],
          [ 774, 2669,  419, 2669,  894,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0],
          [1649, 2076, 1980, 2669,  128,  401, 2669, 2669,  670, 2651,  670, 2651,
            670, 2651, 2669, 2669,  336, 1356, 2669,  525],
          [1374, 2669,  532, 2309,  696,  599,   27, 1721,   64, 1493,  292, 1703,
           1972,  292, 1015,  590, 2611,    0,    0,    0],

In [60]:
#Membuat model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        #embedding (banyaknya keseluruhan kata, 300)
        self.embedding = nn.Embedding(len(word2index), 300)
        #Bergantung pada jumlah embedding sebagai input yaitu 300
        self.lstm = nn.LSTM(300, 512, batch_first=True, num_layers=2)
        #jumlah 512 sama dengan LSTM
        self.fc = nn.Linear(512,2)
        
    def forward(self, x):
        embedded = self.embedding(x)
        #hidden states dan cell
        lstm_out, (hs, cs) = self.lstm(embedded)
        logits = self.fc(hs[0])
        return logits

In [61]:
model = Model()

In [82]:
model(x)

torch.Size([10, 20, 512]) torch.Size([1, 10, 512]) torch.Size([1, 10, 512]) torch.Size([10, 512])


tensor([[-0.0350,  0.1884],
        [-0.0332,  0.1899],
        [-0.0338,  0.1888],
        [-0.0354,  0.1883],
        [-0.0671,  0.1420],
        [ 0.0516,  0.1585],
        [-0.0332,  0.1908],
        [-0.0337,  0.1897],
        [-0.0330,  0.1859],
        [-0.0330,  0.1893]], grad_fn=<AddmmBackward>)

In [46]:
m = nn.LSTM(300, 512, batch_first=True)
m

LSTM(300, 512, batch_first=True)

In [74]:
v = train_batch[0][0]

In [75]:
logits = model(v)

In [76]:
logits

tensor([[ 3.9957, -3.8601],
        [ 4.2232, -4.0898],
        [-4.6268,  4.7387],
        [ 4.0788, -3.9435],
        [ 3.7771, -3.6132],
        [-5.1610,  5.2015],
        [ 4.1543, -3.9773],
        [-4.6005,  4.7002],
        [ 3.3584, -3.2145],
        [ 4.6655, -4.5023]], grad_fn=<AddmmBackward>)

In [51]:
m = nn.Linear(512, 2)
input = torch.randn(128, 512)
output = m(input)
print(output.size())

torch.Size([128, 2])


In [271]:
len(label2index)

2

In [62]:
model = Model()

In [323]:
x = train_batch[0][0]

In [324]:
logits = model(x)

In [325]:
logits

tensor([[-0.2810, -0.0628],
        [-0.2633, -0.0358],
        [-0.2549, -0.0690],
        [-0.2455, -0.0614],
        [-0.2919, -0.0364],
        [-0.2723, -0.0566],
        [-0.2793, -0.0601],
        [-0.2797, -0.0617],
        [-0.2721, -0.0595],
        [-0.2807, -0.0623]], grad_fn=<AddmmBackward>)

In [276]:
logits.shape

torch.Size([10, 2])

In [62]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [327]:
model.embedding.weight

Parameter containing:
tensor([[ 1.3240, -0.2079, -1.1291,  ...,  0.3605, -0.8161,  0.3773],
        [ 0.8693, -2.1738,  0.0808,  ..., -0.5428, -0.3855,  0.7965],
        [-1.4434, -0.0285,  0.8085,  ...,  0.8215, -0.1228, -0.4427],
        ...,
        [-0.6804,  1.2135,  0.9920,  ...,  0.1194, -2.2676,  0.7465],
        [ 0.4419, -0.3777, -0.3510,  ..., -0.2626, -0.6368,  1.4762],
        [ 0.2175, -0.2228, -1.3845,  ..., -0.0035, -0.7638, -0.4752]],
       requires_grad=True)

In [63]:
epochs = 10
for e in range(epochs):
    print('epoch: {}'.format(e+1))
    model.train()
    total_loss = 0
    #x = text, y = label
    for i, batch in enumerate(train_batch):
        x, y = batch
        #validasi batch pada data 
        optimizer.zero_grad()
        #sentimen pada model
        logits = model(x)
        #Menghitung loss
        loss = criterion(logits, y)
        #loss backward dan optimizer iterate pada seluruh parameter, update 
        #nilai grad yang disimpan
        loss.backward()
        optimizer.step()
        #menghitung total loss 
        total_loss += loss.item()
        #menghitung average loss
        avg_loss = total_loss/(i+1)
        
        sys.stdout.write('\riter: {}; avg_loss: {:.3f}'.format(i+1, avg_loss))
    print()

epoch: 1
iter: 72; avg_loss: 0.712
epoch: 2
iter: 72; avg_loss: 0.653
epoch: 3
iter: 72; avg_loss: 0.554
epoch: 4
iter: 72; avg_loss: 0.299
epoch: 5
iter: 72; avg_loss: 0.107
epoch: 6
iter: 72; avg_loss: 0.083
epoch: 7
iter: 72; avg_loss: 0.029
epoch: 8
iter: 72; avg_loss: 0.009
epoch: 9
iter: 72; avg_loss: 0.002
epoch: 10
iter: 72; avg_loss: 0.001


In [ ]:
model(x)

In [123]:
def pred(text):
    token = text.split()
    x = torch.LongTensor([word2index.get(t, word2index['_UNK_']) for t in token])
    x = x.unsqueeze(0)
    y = model(x)[0]
    y = y.argmax(dim=-1).item()
    if y == 1:
        sentiment = 'positive'
    else:
        sentiment ='negative'
    result = {}
    for i in ['text', 'sentiment']:
        result[i] = eval(i)
    print(result)

In [124]:
pred('Tidak suka dengan pemilu')

{'text': 'Tidak suka dengan pemilu', 'sentiment': 'negative'}


In [125]:
pred('Semoga pemilu kali ini lancar')

{'text': 'Semoga pemilu kali ini lancar', 'sentiment': 'negative'}


In [126]:
pred('Golput sajalahh')

{'text': 'Golput sajalahh', 'sentiment': 'negative'}


In [83]:
model.eval

<bound method Module.eval of Model(
  (embedding): Embedding(2671, 300)
  (lstm): LSTM(300, 512, num_layers=2, batch_first=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)>

In [102]:
x = torch.ones(1, requires_grad=True)
print(x)

tensor([1.], requires_grad=True)
